In [26]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import pandas as pd

In [27]:
# Spark Constants
APP_NAME = 'assignment1'
MASTER = 'local[*]'

#random hyper planes
# Data Columns
COLUMN_USER_ID = 'userId'
COLUMN_MOVIE_ID = 'movieId'
COLUMN_RATING = 'rating'
COLUMN_TIMESTAMP = 'timestamp'

# Input Constants
MOVIES_INPUT_FILE = 'movies.csv'
TAGS_INPUT_FILE = 'tags.csv'
RATINGS_INPUT_FILE = 'ratings.csv'

In [28]:
conf = SparkConf().setAppName(APP_NAME).setMaster(MASTER)
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession.builder.appName(APP_NAME).master(MASTER).getOrCreate()

In [25]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering

ratings = pd.read_csv('ratings.csv')

user_item_matrix = ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)

X = user_item_matrix.values
X = StandardScaler().fit(X).transform(X)

clusters = AgglomerativeClustering(n_clusters=None, distance_threshold=200).fit_predict(X)

print(clusters)
print(clusters.shape)

[ 5 16  8 ... 21 21 21]
(9724,)


In [212]:
import numpy as np

from numpy import dot
from numpy.linalg import norm

def predict_rating(user_id, movie_id):
    movie_idx = user_item_matrix.index.get_loc(movie_id)
    movie_to_predict = user_item_matrix.loc[movie_id]

    similar_movies = user_item_matrix[clusters == clusters[movie_idx]].drop(movie_id)

    distances = dot(movie_to_predict, similar_movies.T)/(norm(movie_to_predict)*norm(similar_movies.T))
    
    user_ratings = similar_movies.get(user_id)

    rating_prediction = np.dot(user_ratings, distances).sum()/distances[user_ratings != 0].sum()

    return rating_prediction

In [211]:
predict_rating(416, 319)

movieId
327     4.5
531     2.5
1199    4.0
3624    2.0
4361    1.0
Name: 416, dtype: float64


3.2368148914167527